In [1]:
import torch
from transformers import AutoModel, AutoTokenizer

# Check if GPU is available
print("CUDA available:", torch.cuda.is_available())

# Load a model (will automatically use GPU if available)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# Move to GPU if available
if torch.cuda.is_available():
    model = model.cuda()

# Test inference
text = "Hello, this is a test sentence."
inputs = tokenizer(text, return_tensors="pt")

# Move inputs to GPU
if torch.cuda.is_available():
    inputs = inputs.to("cuda")

# Run model
with torch.no_grad():
    outputs = model(**inputs)

print("Model output shape:", outputs.last_hidden_state.shape)
print("Running on:", "GPU" if outputs.last_hidden_state.device.type == "cuda" else "CPU")

/home/capybara/Desktop/matura_project_python/HFvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
Model output shape: torch.Size([1, 10, 768])
Running on: GPU
